# Notebeook version of NORTRIP_Control.py
Notebook version of Nortrip-pythonfor easier visualization and testing.

### Arguments

In [ ]:
read_as_text = False
print_results = False
use_fortran = False
use_logging = False
plot_figure = [0,0,0,0,0,0,0,0,0,0,0,0,1,0]
model_paths = "../model_paths/model_paths_and_files.xlsx"

### Imports

In [ ]:
import os
from read_files import (
    read_road_dust_parameters,
    read_road_dust_paths,
    read_road_dust_input,
)
from initialise import (
    road_dust_initialise_time,
    convert_road_dust_input,
)
import logging
from fortran import NORTRIP_fortran_control
from plots import plot_road_dust_result
from output import save_road_dust_results_average
from main_nortrip_loop import main_nortrip_loop

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

### Read file paths and parameters

In [ ]:
paths = read_road_dust_paths(read_as_text=read_as_text, paths_xlsx=model_paths)

model_parameters, model_flags, model_activities, parameter_sheets = (
    read_road_dust_parameters(
        paths.path_filename_inputparam, read_as_text=read_as_text
    )
)

### Read input data

In [ ]:
input_data = read_road_dust_input(
        paths.path_filename_inputdata,
        model_parameters,
        read_as_text,
    )

activity_input = input_data.activity
airquality_input = input_data.airquality
meteorology_input = input_data.meteorology
traffic_input = input_data.traffic
initial_input = input_data.initial
metadata_input = input_data.metadata

### Convert input data and initialise time

In [ ]:
converted_data = convert_road_dust_input(input_data, nodata=metadata_input.nodata)

time_config = road_dust_initialise_time(
    converted_data=converted_data,
    metadata=metadata_input,
    use_fortran_flag=use_fortran,
)

if time_config.time_bad:
    logger.error("Time configuration failed - stopping execution")
    exit()

### Run Fortran model
(WIP) Does nothing as of now.

In [ ]:
if use_fortran:
    NORTRIP_fortran_control()

#### Main model loop

In [ ]:
model_variables = main_nortrip_loop(
    time_config=time_config,
    converted_data=converted_data,
    metadata_input=metadata_input,
    initial_input=initial_input,
    model_flags=model_flags,
    model_parameters=model_parameters,
    meteorology_input=meteorology_input,
    traffic_input=traffic_input,
    activity_input=activity_input,
    airquality_input=airquality_input,
    model_activities=model_activities,
)

### Plot results
Plots the plots defined in the arguments section of the notebook.

Will save plots if save_type_flag is 2 or 3.

In [ ]:
time_config.min_time = time_config.min_time_save
time_config.max_time = time_config.max_time_save

min_time_original = time_config.min_time
max_time_original = time_config.max_time

# Plots are saved if save_type_flag is 2 or 3
save_plots = model_flags.save_type_flag in [2, 3]
if save_plots:
    os.makedirs(paths.path_outputfig, exist_ok=True)
    logger.info(f"Creating and saving plots to {paths.path_outputfig}...")

plot_road_dust_result(
    time_config=time_config,
    converted_data=converted_data,
    metadata=metadata_input,
    airquality_data=airquality_input,
    model_parameters=model_parameters,
    model_flags=model_flags,
    model_variables=model_variables,
    meteo_input=meteorology_input,
    input_activity=activity_input,
    paths=paths,
    ro=0,
    plot_figure=plot_figure,
    print_result=print_results,
    save_plots=save_plots,
)

time_config.min_time = min_time_original
time_config.max_time = max_time_original

### Save results

Will save results if save_type_flag is 1, 3 or 4.

Will save as text if save_type_flag is 4.

In [ ]:
save_data = model_flags.save_type_flag in [1, 3, 4]
save_as_text = model_flags.save_type_flag == 4

if save_data:
    save_road_dust_results_average(
        time_config=time_config,
        converted_data=converted_data,
        metadata=metadata_input,
        airquality_data=airquality_input,
        model_parameters=model_parameters,
        model_flags=model_flags,
        model_variables=model_variables,
        input_activity=activity_input,
        av=[model_flags.plot_type_flag],
        save_as_text=save_as_text,
        paths=paths,
        parameter_sheets=parameter_sheets,
    )

logger.info("End of NORTRIP_Control")